## Todo: Parse Solution is our homework

In [1]:
solution = {}

In [2]:
from functools import lru_cache

In [3]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    # 如果字符串1的长度为0， 则编辑距离为字符串2的长度
    if len(string1) == 0: 
        return len(string2)
    
    # 如果字符串2的长度为0， 则编辑距离为字符串1的长度
    if len(string2) == 0: 
        return len(string1)
    
    # 字符串1 最后的字符
    tail_s1 = string1[-1]
    
    # 字符串2 最后的字符
    tail_s2 = string2[-1]
    
    candidates = [
        # 字符串1末尾删除一个和字符串2末尾字符相等的字符，编辑距离+1
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        
        # 字符串1最后添加字符串2的一个字符，编辑距离+1
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
    ]
    
    # 字符串1和字符串2最后的字符相等，编辑距离不变
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    
    # 如果不相同，则编辑距离+1
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    # 增加，删除，替换 三种操作放在一起比较
    candidates.append(both_forward)
    
    # 选出最小编辑距离的操作
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    # 存入操作方式中
    solution[(string1, string2)] = operation 
    
    return min_distance

In [4]:
string1 = 'ABCDECG'
string2 = 'ABCCEF'
edit_distance(string1, string2)

3

In [5]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'A'): 'DEL G',
 ('ABCDECG', 'A

In [6]:
result = []

def parse_solution(str1, str2):
    operation = solution[(str1, str2)]
    print(str1, str2, operation)

    if str1 == str2:
        return result
    
    else:
        op = operation.split(' ')[0]
        if op =='ADD':
            _str1, _str2 = str1, str2[:-1]
        if op == 'DEL':
            _str1, _str2 = str1[:-1], str2
        else:
            _str1, _str2 = str1[:-1], str2[:-1]
        
        result.append(operation)

    return parse_solution(_str1, _str2)

In [7]:
parse_solution(string1, string2)

ABCDECG ABCCEF DEL G
ABCDEC ABCCEF SUB C => F
ABCDE ABCCE 
ABCD ABCC SUB D => C
ABC ABC 


['DEL G', 'SUB C => F', '', 'SUB D => C']

# Problem Case 3: Pinyin Auto Correction Problem

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

In [1]:
from collections import Counter
import pinyin


# 读取词库数据
chinese_dataset = 'article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset).read()
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [5]:
# 汉字转为拼音，并且全部转为小写
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS).lower()

In [6]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [7]:
import re

def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

# 去除所有数字，并分词
PINYIN = tokens(CHINESE_CHARATERS_COPYS)

In [10]:
PINYIN[:10]

['ci', 'wai', 'zi', 'ben', 'zhou', 'yue', 'ri', 'qi', 'chu', 'xiao']

In [12]:
# 根据分词，获取前一个拼音的后一个字母，和后一个拼音的前一个字母，并组合成2个字母的分词标志
letter_cut = [PINYIN[i][-1] + PINYIN[i+1][0] for i in range(len(PINYIN)-1)]
letter_cut[:10]

In [14]:
cut_count = Counter(letter_cut)
cut_count

In [37]:
# 根据分词，获取前一个拼音的后2个字母，和后一个拼音的前2个字母，并组合成4个字母的分词标志
letter_cut_2 = [PINYIN[i][-2:] + PINYIN[i+1][:2] for i in range(len(PINYIN)-1)]
letter_cut_2[:10]

['ciwa',
 'aizi',
 'zibe',
 'enzh',
 'ouyu',
 'ueri',
 'riqi',
 'qich',
 'huxi',
 'aomi']

In [38]:
# 获取count
cut_count_2 = Counter(letter_cut_2)
cut_count_2

Counter({'ciwa': 4749,
         'aizi': 21919,
         'zibe': 4139,
         'enzh': 58172,
         'ouyu': 21087,
         'ueri': 165398,
         'riqi': 8206,
         'qich': 11984,
         'huxi': 38468,
         'aomi': 11640,
         'mish': 3894,
         'ouji': 53059,
         'jide': 19083,
         'ngku': 23288,
         'anji': 261309,
         'jixi': 20389,
         'ngwa': 33728,
         'aiqi': 21708,
         'qiyu': 3864,
         'yuji': 23935,
         'ngyi': 141680,
         'yiza': 5898,
         'anti': 32981,
         'ngge': 38719,
         'ngxi': 264524,
         'infa': 14361,
         'fabu': 13302,
         'buha': 1788,
         'anka': 28338,
         'aifa': 33338,
         'faba': 964,
         'tiya': 4646,
         'anba': 41452,
         'anne': 13681,
         'eice': 1761,
         'cewe': 496,
         'endi': 16853,
         'ngba': 53152,
         'anza': 45449,
         'anbu': 42646,
         'bush': 23020,
         'ouyi': 34587,
 

In [103]:
def cut_pinyin(text):
    cut_point = []

    # 分词概率，大于该概率表示分词
    p = 10e-7

    # 从第一个字母和第二个字母之间的位置开始

    print('拼音句子', text)
    for i in range(1, len(text) - 1):

    #     first, second = test_str[:i][-1:], test_str[i:][:1]
    #     if first+second in cut_count and cut_count[first+second]/len(letter_cut) > p:
    #         print(first+second)
    #         cut_point.append(i)

        # 获取分词点的前两个字母 和 后两个字母
        previous_two, next_two = text[:i][-2:], text[i:][:2]

        # 组合成四个字母的分词标志，从字典中获取分词标志的概率
        if previous_two+next_two in cut_count_2 and cut_count_2[previous_two+next_two]/len(letter_cut_2) > p:
            print(previous_two, '|' , next_two, '分词点', i)
            cut_point.append(i)

    res = [text[i:j] for i, j in zip([0]+cut_point, cut_point+[None])]
    return res 

In [105]:
cut_pinyin('houchangligongxueyuan')  # 不能识别 yuan

拼音句子 houchangligongxueyuan
ou | ch 分词点 3
ng | li 分词点 8
li | go 分词点 10
ng | xu 分词点 14
ue | yu 分词点 17
yu | an 分词点 19


['hou', 'chang', 'li', 'gong', 'xue', 'yu', 'an']